In [2]:
# Run ExtractPhyloWGSResults.py and Rscripts/ProcessBSCITEResults.R.

from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import v_measure_score

import pandas as pd
import numpy as np
import os

data_path = "../_output/HGSOC_10X/"

import sklearn
sklearn.__version__

'0.23.1'

In [9]:
gt = pd.read_csv("../data/HGSOC_10X/HGSOC_10X_gt.txt", header=0, sep=" ")
valid_clone_names = ["A_B_C_D_E_F_G_H_I", "A_B_C_D", "A_B", "C_D", "A", "B", "C", "D", "E_F_G_H_I", "E_F", "E", "F"]

# Evaluate the ancestral metric.
# Get the true ancestral metric: not many SNVs, just do a plain double for loops. 
snv_count = gt.shape[0]
A = np.zeros(shape = (snv_count, snv_count))
for i in range(snv_count):
    clone_i = set(gt.iloc[i]["CloneName"].split("_"))
    for j in range(snv_count):
        clone_j = set(gt.iloc[j]["CloneName"].split("_"))
        if clone_i != clone_j and clone_j.issubset(clone_i):
            A[i,j] = 1

idx = np.array(np.where(gt["CloneName"].isin(valid_clone_names)))[0]
valid_idx = np.ix_(idx, idx)
A0 = A[valid_idx]


93

In [10]:
# Our method.
chain_count = 4
metrics = np.zeros([chain_count, 4])
for chain in range(chain_count):
    chain_path = "../_output/HGSOC_10X/phylex/chain" + str(chain)
    predicted = pd.read_csv(chain_path + "/joint/tree0/cluster_labels.tsv", header=None, sep="\t", names=["ID", "CloneName"])
    tbl_join = predicted.join(gt, lsuffix='_caller', rsuffix='_other')
    ret = tbl_join[tbl_join["CloneName_other"].isnull() == False]
    ret_valid = tbl_join[tbl_join["CloneName_other"].isin(valid_clone_names)]

    ancestral_matrix = pd.read_csv(chain_path + "/joint/tree0/ancestral_matrix.csv", header=None)
    ancestral_matrix = np.asarray(ancestral_matrix)
    ancestral_matrix_0 = ancestral_matrix[valid_idx]
    metrics[chain,0] = v_measure_score(ret_valid["CloneName_other"], ret_valid["CloneName_caller"])
    metrics[chain,1] = adjusted_rand_score(ret_valid["CloneName_other"], ret_valid["CloneName_caller"])
    metrics[chain,2] = adjusted_mutual_info_score(ret_valid["CloneName_other"], ret_valid["CloneName_caller"])
    metrics[chain,3] = np.mean(np.abs(ancestral_matrix_0 - A0))
    
# V-measure, adjusted rand score, adjusted mutual info, ancestral metric.
print(metrics.mean(0))


[0.33340196 0.19993513 0.24303573 0.23184884]
[0.02687659 0.01159452 0.02301161 0.00678741]


In [11]:
# TSSB

In [12]:
# Canopy